In [10]:
import esm
from esm.sdk.api import ESMProtein, GenerationConfig
import py3Dmol
import biotite.structure.io.pdbx as pdbx
from biotite.structure.io.pdb import PDBFile
from biotite.structure import superimpose, rmsd
from io import StringIO
import warnings

You need an account on: https://forge.evolutionaryscale.ai/

After you have made an account, log in, click on "API Keys" and create a new key.

In [ ]:
from getpass import getpass

# Enter your API key here. Remeber you need to keep it confidential.
# Never add your key to a github repo or post it publicly in any way.
api_key = getpass("Your EvolutionaryScale API key: ")


In [42]:
model = esm.sdk.client("esm3-medium-2024-08", token=api_key)

To fold the protein you have created, go to: https://alphafoldserver.com/

You need to set up an account there as well.

In [3]:
def mask_sequence(protein, retain=None, replace=None):
    """
    Mask parts of a protein sequence with underscores.
    
    Parameters:
    -----------
    protein : str
        The amino acid sequence
    retain : str, optional
        Positions to retain (1-indexed), e.g., "1,2,6-10,22-45"
    replace : str, optional
        Positions to replace (1-indexed), e.g., "1-4,15,20-25"
    
    Returns:
    --------
    str
        The masked sequence
    
    Note: If both retain and replace are specified, retain takes precedence.
    """
    def parse_positions(pos_string):
        positions = set()
        for part in pos_string.split(','):
            if '-' in part:
                start, end = map(int, part.split('-'))
                positions.update(range(start, end + 1))
            else:
                positions.add(int(part))
        return positions
    
    masked = list(protein)
    
    if retain:
        keep = parse_positions(retain)
        masked = [ch if i+1 in keep else '_' for i, ch in enumerate(masked)]
    elif replace:
        rep = parse_positions(replace)
        masked = ['_' if i+1 in rep else ch for i, ch in enumerate(masked)]
    
    return ''.join(masked)


# Example usage:
protein = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQC"

# Example 1: Retain specific positions
result1 = mask_sequence(protein, retain="1,2,6-10,22-45")
print("Retain 1,2,6-10,22-45:")
print(result1)

# Example 2: Replace specific positions (e.g., replace first 4)
result2 = mask_sequence(protein, replace="1-4")
print("\nReplace 1-4:")
print(result2)

# Example 3: Retain positions 5-15
result3 = mask_sequence(protein, retain="5-15")
print("\nRetain 5-15:")
print(result3)

# Example 4: Replace multiple ranges
result4 = mask_sequence(protein, replace="1-10,20-30,50-60")
print("\nReplace 1-10,20-30,50-60:")
print(result4)

Retain 1,2,6-10,22-45:
MS___ELFTG___________VNGHKFSVSGEGEGDATYGKLTLK_________________________

Replace 1-4:
____EELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQC

Retain 5-15:
____EELFTGVVPIL_______________________________________________________

Replace 1-10,20-30,50-60:
__________VVPILVELD___________GEGEGDATYGKLTLKFICT___________VTTFSYGVQC


In [8]:
# Visualize the masked regions
view = py3Dmol.view(width=600, height=400)
view.addModel(open('GFP_AF3.cif').read(), 'cif')

view.setStyle({}, {})  # hide everything
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'lightblue'}})
view.setStyle({'chain': 'A', 'resi': ['129-147', '188-198']}, {'cartoon': {'color': 'tomato'}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
# GFP
# Sequence: https://www.uniprot.org/uniprotkb/P42212/
# Structure: https://www.rcsb.org/3d-view/1B9C/1
protein = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
masked = mask_sequence(protein, replace="129-147,188-198")
print(masked)

MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGI___________________HNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTP___________HYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


In [ ]:
if False:
    protein = ESMProtein(sequence=masked)

    # Generate the protein sequence
    protein = model.generate(
        protein,
        GenerationConfig(
            track="sequence",  # specify that we want to generate a sequence
            num_steps=8,      # number of steps for iterative decoding
            temperature=0.7   # controls diversity vs. perplexity
        )
    )

    # The `protein` object now contains the generated sequence.
    # You can access it via `protein.sequence`
    print(protein.sequence)

MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIGFKKGGKILDGKLIKDIEPHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPLGNGKLPVLPPHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


In [24]:
# suppress irrelevant warnings
warnings.filterwarnings('ignore', message='Attribute .* not found within')

def align_and_visualize_cif(cif_file1, cif_file2, chain1='A', chain2='A', highlight=None):
    """
    Align two CIF files and visualize the overlay.
    
    Parameters:
    -----------
    cif_file1 : str
        Reference CIF file
    cif_file2 : str
        CIF file to align
    chain1, chain2 : str
        Chain IDs to use for alignment
    """
    # Read CIF files
    structure1 = pdbx.get_structure(pdbx.CIFFile.read(cif_file1), model=1)
    structure2 = pdbx.get_structure(pdbx.CIFFile.read(cif_file2), model=1)
    
    # Get CA atoms for alignment
    ca1 = structure1[(structure1.chain_id == chain1) & (structure1.atom_name == 'CA')]
    ca2 = structure2[(structure2.chain_id == chain2) & (structure2.atom_name == 'CA')]
    
    # Match lengths
    n_atoms = min(len(ca1), len(ca2))
    ca1 = ca1[:n_atoms]
    ca2 = ca2[:n_atoms]
    
    # Superimpose CA atoms to get transformation
    ca2_aligned, transformation = superimpose(ca1, ca2)
    
    # Apply transformation to full structure
    structure2_aligned = structure2.copy()
    structure2_aligned.coord = transformation.apply(structure2.coord)
    
    # Print alignment info
    print(f"RMSD: {rmsd(ca1, ca2_aligned):.2f} Å ({n_atoms} CA atoms)")
    
    # Convert to PDB format for visualization
    pdb1 = StringIO()
    pdb_file1 = PDBFile()
    pdb_file1.set_structure(structure1)
    pdb_file1.write(pdb1)
    
    pdb2 = StringIO()
    pdb_file2 = PDBFile()
    pdb_file2.set_structure(structure2_aligned)
    pdb_file2.write(pdb2)
    
    # Visualize
    view = py3Dmol.view(width=600, height=400)
    view.addModel(pdb1.getvalue(), 'pdb')
    view.addModel(pdb2.getvalue(), 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'lightblue', 'opacity': .8}})
    view.setStyle({'model': 1}, {'cartoon': {'color': 'khaki'}})
    if highlight is not None:
        view.setStyle({'model': 1, 'resi': highlight}, {'cartoon': {'color': 'tomato'}})
    view.zoomTo()
    
    return view

# Usage
view = align_and_visualize_cif('GFP_AF3.cif', 'tmp/fold_2025_11_02_01_36_model_4.cif', highlight = ['129-147', '188-238'])
view.show()

RMSD: 5.81 Å (238 CA atoms)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [28]:
# Usage
view = align_and_visualize_cif('tmp/fold_2025_11_02_01_36_model_1.cif', 'tmp/fold_2025_11_02_01_36_model_4.cif')
view.show()

RMSD: 1.06 Å (238 CA atoms)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Exercises

- Mask different parts of the GFP sequence and see what results you get.

- See how much you can delete from the GFP sequence before results start falling apart. For example, what happens when you mask from position 188 all the way till the end (position 238)?